In [6]:
#! /usr/bin/env python

import covasim as cv
import numpy as np
import pandas as pd
from os.path import join
import matplotlib.pyplot as plt
import seaborn as sns
import sciris as sc
from datetime import datetime, timedelta


contact_dir = '/mnt/d/books/iitm/agentBased/codes/tn_contact_matrix/output/contact_matrix'
home_path = join(contact_dir, 'home_contact.csv')
school_path = join(contact_dir, 'school_contact.csv')
work_path = join(contact_dir, 'work_contact.csv')
community_path = join(contact_dir, 'community_contact.csv')


fb_dir = '/mnt/d/books/iitm/agentBased/codes/covasim/models/data/'
tile_path = join(fb_dir, 'tn_quadkey.csv')
density_path = join(fb_dir, 'pop_density.csv')
mobility_path = '/mnt/d/books/iitm/agentBased/data/fb/mobility/'

refactor_file = '/mnt/d/books/iitm/agentBased/data/tn/incovid19/covasim/tmp_datafile.csv'

start_day, end_day = '2020-06-01', '2020-06-30'

asymptotic_rate = 0.3
pop_infected = (22333 - 12757) * (1 + asymptotic_rate)

age_based_infection = [0.045455, 0.009091, 0.014773, 0.00000, 0.051136, 0.022727, 0.068182, 0.102273, 0.211364, 0.029924, 0.120833, 0.117424, 0.04697, 0.063636, 0.000000, 0.096212]
tile_infection = [0.        , 0.00430416, 0.00215208, 0.04925873, 0.00023912,
       0.00454328, 0.03778097, 0.02223816, 0.00621712, 0.15845688,
       0.01721664, 0.17663   , 0.14506616, 0.01602104, 0.0215208 ,
       0.0143472 , 0.04710665, 0.00701419, 0.04949785, 0.05499761,
       0.02662203, 0.01960784, 0.01259366, 0.00557947, 0.01769488,
       0.0263032 , 0.00557947, 0.00765184, 0.02941176, 0.0143472 ,
       0.        ]


pars = dict(
    start_day = start_day,
    end_day = end_day,

    pop_type = 'matrix', # Use a more realistic population model
    location = 'India-TamilNadu', # Use population characteristics for Tamil-Nadu

    pop_size = 50_000,
    pop_scale = 2,
    rescale = True,
    pop_infected = pop_infected,

    home_matrix=home_path,
    school_matrix = school_path,
    work_matrix = work_path,
    community_matrix = community_path,

    tiles = tile_path,
    pop_density = density_path,
    mobility = mobility_path,

    use_waning = True,
    dynam_layer={'c': True},

    init_infection={'tiles':tile_infection, 'ages':age_based_infection},

    rand_seed=2
)


good_esti = {}
good_esti['beta'] = 0.015
good_esti['rel_death_prob'] = 1.0
good_esti['work_changes'] = 0.3
good_esti['community_changes'] = 0.3


def get_sim(good_esti):
  close_school = cv.clip_edges(days=0, changes=0, layers='s')
  reduce_work  = cv.clip_edges(days=0, changes=good_esti['work_changes'], layers='w')
  reduce_community = cv.clip_edges(days=0, changes=good_esti['community_changes'], layers='c')
  testing = cv.test_num(daily_tests='data')
  sim = cv.Sim(pars=pars, datafile=refactor_file, interventions=[close_school, reduce_work, reduce_community, testing])
  return sim


def create_data_file(start, end):
  case_datafile = '/mnt/d/books/iitm/agentBased/data/tn/incovid19/covasim/state_datafile.csv'
  refactor_file = '/mnt/d/books/iitm/agentBased/data/tn/incovid19/covasim/tmp_datafile.csv'
  df = pd.read_csv(case_datafile, delimiter=',')
  df = df[(df['date']>=start) & (df['date']<=end)]
  df.to_csv(refactor_file, sep=',', index=False, header=True)


def run_week(sim, good_esti):
  calib_pars = dict(
    beta           = [good_esti['beta'], 0.005, 0.60],
    rel_death_prob = [good_esti['rel_death_prob'], 0.5, 4.0],
    work_changes = [good_esti['work_changes'], 0.0, 1.0],
    community_changes = [good_esti['community_changes'], 0.0, 1.0],
  )

  fit_args = dict(
    weights = {'new_diagnoses': 0.25, 'new_recoveries': 0.25 , 'new_deaths': 0.5},
    keys = ['new_diagnoses', 'new_recoveries', 'new_deaths'],
    normalize=False,
    use_squared=True,
    as_scalar='mean'
  )

  def set_clip_edges(sim, calib_pars):
    _, work, community = sim.get_interventions(cv.clip_edges)
    work.changes = calib_pars['work_changes']
    community.changes = calib_pars['community_changes']
    return sim

  n_trials = 1
  n_workers = 1

  calib = sim.calibrate(calib_pars=calib_pars, custom_fn=set_clip_edges, fit_args=fit_args, n_trials=n_trials, n_workers=n_workers)

  print(f"start: {calib.after.pars['start_day']} end: {calib.after.pars['end_day']}")
  calib.summarize()

  return calib


def run():
  start = start_day
  weeks = 2
  calib = None
  for w in range(weeks):
    end_d = datetime.strptime(start, '%Y-%m-%d') + timedelta(days=6)
    end = end_d.strftime('%Y-%m-%d')
    pars['start_day'] = start
    pars['end_day'] = end
    create_data_file(start, end)
    if w != 0:
      good_esti['beta'] = calib.best_pars['beta']
      good_esti['rel_death_prob'] = calib.best_pars['rel_death_prob']
      good_esti['work_changes'] = calib.best_pars['work_changes']
      good_esti['community_changes'] = calib.best_pars['community_changes']
      pars['pop_infected'] = calib.after.scaled_pop_size - calib.after.results['n_susceptible'][-1]
    sim = get_sim(good_esti)
    calib = run_week(sim, good_esti)


run()

  Automatically adding cumulative column cum_tests from new_tests
  Automatically adding cumulative column cum_diagnoses from new_diagnoses
  Automatically adding cumulative column cum_recoveries from new_recoveries
  Automatically adding cumulative column cum_deaths from new_deaths


[I 2022-04-14 12:10:49,162] A new study created in RDB with name: covasim_calibration


beta 0.005 0.6 0.015
rel_death_prob 0.5 4.0 1.0
work_changes 0.0 1.0 0.3
community_changes 0.0 1.0 0.3
Initializing sim with 50000 people for 6 days
Loading location-specific data for "India-TamilNadu"
  Running 2020-06-01 ( 0/6) (2.21 s)  ••—————————————————— 14%


[I 2022-04-14 12:10:57,379] Trial 0 finished with value: 276087.0714285714 and parameters: {'beta': 0.5770541852484938, 'rel_death_prob': 2.1298276236407006, 'work_changes': 0.7202368480677805, 'community_changes': 0.7400692015196513}. Best is trial 0 with value: 276087.0714285714.


Simulation summary:
       48,626 cumulative infections
            2 cumulative reinfections
       10,436 cumulative infectious
        5,118 cumulative symptomatic cases
           30 cumulative severe cases
            2 cumulative critical cases
            6 cumulative recoveries
            0 cumulative deaths
      101,008 cumulative tests
        6,626 cumulative diagnoses
            0 cumulative known deaths
            0 cumulative quarantined people
            0 cumulative vaccine doses
            0 cumulative vaccinated people

Initializing sim with 50000 people for 6 days
Loading location-specific data for "India-TamilNadu"
  Running "Before calibration": 2020-06-01 ( 0/6) (2.13 s)  ••—————————————————— 14%
Simulation "Before calibration" summary:
       26,908 cumulative infections
            0 cumulative reinfections
        8,242 cumulative infectious
        4,658 cumulative symptomatic cases
           30 cumulative severe cases
            2 cumulative critical 

[I 2022-04-14 12:11:13,745] A new study created in RDB with name: covasim_calibration


beta 0.005 0.6 0.5770541852484938
rel_death_prob 0.5 4.0 2.1298276236407006
work_changes 0.0 1.0 0.7202368480677805
community_changes 0.0 1.0 0.7400692015196513
Initializing sim with 50000 people for 6 days
Loading location-specific data for "India-TamilNadu"
  Running 2020-06-01 ( 0/6) (2.31 s)  ••—————————————————— 14%
Simulation summary:
       75,354 cumulative infections
            2 cumulative reinfections
       15,060 cumulative infectious
        8,148 cumulative symptomatic cases
           50 cumulative severe cases
            4 cumulative critical cases
           10 cumulative recoveries
            0 cumulative deaths
      101,008 cumulative tests
       10,308 cumulative diagnoses
            0 cumulative known deaths
            0 cumulative quarantined people
            0 cumulative vaccine doses
            0 cumulative vaccinated people



[I 2022-04-14 12:11:22,650] Trial 0 finished with value: 457958.9285714286 and parameters: {'beta': 0.06036585029699441, 'rel_death_prob': 0.8268792881018963, 'work_changes': 0.33900841625745504, 'community_changes': 0.22755090300455771}. Best is trial 0 with value: 457958.9285714286.


Initializing sim with 50000 people for 6 days
Loading location-specific data for "India-TamilNadu"
  Running "Before calibration": 2020-06-01 ( 0/6) (2.54 s)  ••—————————————————— 14%
Simulation "Before calibration" summary:
       98,526 cumulative infections
            4 cumulative reinfections
       18,158 cumulative infectious
        8,950 cumulative symptomatic cases
           50 cumulative severe cases
            4 cumulative critical cases
           10 cumulative recoveries
            0 cumulative deaths
      101,006 cumulative tests
       11,420 cumulative diagnoses
            0 cumulative known deaths
            0 cumulative quarantined people
            0 cumulative vaccine doses
            0 cumulative vaccinated people

Initializing sim with 50000 people for 6 days
Loading location-specific data for "India-TamilNadu"
  Running "After calibration": 2020-06-01 ( 0/6) (2.80 s)  ••—————————————————— 14%
Simulation "After calibration" summary:
       75,354 cumulati